In [ ]:
import lightgbm as lgb
import numpy as np
import pandas as pd
import numpy as np
import gc
import os
import optuna
import sqlite3
import ray
import matplotlib.pyplot as plt
from optuna.integration import LightGBMPruningCallback
from autogluon.tabular import TabularPredictor
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Rutas de entrada y salida
input_path = './data/l_vm_completa_normalizada_fe.parquet'
output_train = './data/df_train.parquet'
output_val = './data/df_val.parquet'

# Periodos para división
periodo_train_max = 201908
periodos_val = [201909, 201910]

In [ ]:


# Leer datos completos (solo una vez) y filtrar por los periodos necesarios
print("Leyendo y filtrando datos...")
df = pd.read_parquet(input_path, engine='fastparquet')

# Filtrar en dos partes directamente sin copias innecesarias
df_train = df[df['PERIODO'] <= periodo_train_max]
df_val = df[df['PERIODO'].isin(periodos_val)]

# Liberar el DataFrame original
del df
gc.collect()

# Guardar a disco
print("Guardando conjuntos en disco...")
df_train.to_parquet(output_train, index=False)
df_val.to_parquet(output_val, index=False)

# Liberar memoria final
del df_train, df_val
gc.collect()

print("✅ Proceso completado.")
print(f" - df_train → {output_train}")
print(f" - df_val   → {output_val}")


In [2]:
df_train = pd.read_parquet(output_train, engine='fastparquet')
df_val = pd.read_parquet(output_val, engine='fastparquet')

In [ ]:
import datetime
import gc
from autogluon.tabular import TabularPredictor

gc.collect()

# === Configuración general ===
label_column = 'CLASE_DELTA_ZSCORE'
output_path = f"AutogluonModels/model_fast_{datetime.datetime.now().strftime('%Y%m%d_%H%M')}"

# === Entrenamiento ===
predictor = TabularPredictor(
    label=label_column,
    problem_type='regression',
    eval_metric='mean_absolute_error',
    path=output_path
).fit(
    train_data=df_train,
    tuning_data=df_val,
    use_bag_holdout=True,
    presets='medium_quality',               # Entrenamiento rápido pero razonable
    time_limit=1800,
    num_bag_folds=3,                        # Reduce cantidad de folds
    num_bag_sets=1,                         # Una sola pasada
    auto_stack=False,                       # Apagar stacking para acelerar
    ag_args_ensemble={'fold_fitting_strategy': 'parallel_local'},
    hyperparameters={
        'GBM': [
             {
                'ag_args': {'name_suffix': 'Linear Tree'},
                'linear_tree': True,               
                'early_stopping_rounds': 10,
            },
            {
                'ag_args': {'name_suffix': 'Extra Trees'},
                'extra_trees': True,               
                'early_stopping_rounds': 10,
            },
            {
                'ag_args': {'name_suffix': 'QuickDefault'},
                'num_leaves': 31,
                'feature_fraction': 0.8,
                'bagging_fraction': 0.8,                
                'early_stopping_rounds': 10,
            }
        ]
    },
    verbosity=2
)


In [5]:
predictor.fit_extra(
    time_limit=3600,
    ag_args_ensemble={'fold_fitting_strategy': 'sequential_local', 'num_bag_folds': 1},
    hyperparameters={
        'NN_TORCH': [{
            'num_epochs': 4,
            'learning_rate': 0.01,
            'dropout_prob': 0.1,
            'weight_decay': 1e-5,
            'batch_size': 256,
            'hidden_size': 128,
            'ag_args': {'name_suffix': 'NN_LightFast'},
            'ag_args_fit': {'num_gpus': 1}
        }]
    },
    fit_weighted_ensemble=True,
    verbosity=2
)


Fitting 1 L1 models, fit_strategy="sequential" ...
Fitting model: NeuralNetTorchNN_LightFast_BAG_L1 ... Training model for up to 3600.00s of the 3600.00s of remaining time.
Fitting model: NeuralNetTorchNN_LightFast_BAG_L1 ... Training model for up to 3600.00s of the 3600.00s of remaining time.
	Fitting 3 child models (S1F1 - S1F3) | Fitting with SequentialLocalFoldFittingStrategy
	Fitting 3 child models (S1F1 - S1F3) | Fitting with SequentialLocalFoldFittingStrategy
	-0.1002	 = Validation score   (-mean_absolute_error)
	1398.41s	 = Training   runtime
	37.94s	 = Validation runtime
Fitting model: WeightedEnsemble_2_L2 ... Training model for up to 360.00s of the 1930.10s of remaining time.
	-0.1002	 = Validation score   (-mean_absolute_error)
	1398.41s	 = Training   runtime
	37.94s	 = Validation runtime
Fitting model: WeightedEnsemble_2_L2 ... Training model for up to 360.00s of the 1930.10s of remaining time.
	Ensemble Weights: {'NeuralNetTorchNN_LightFast_BAG_L1': 1.0}
	-0.1002	 = Valid

In [6]:
predictor.fit_extra(
    time_limit=900,  
    ag_args_ensemble={'fold_fitting_strategy': 'parallel_local'},
    fit_weighted_ensemble=True,
    hyperparameters={
        'CAT': [{
            'iterations': 500,
            'depth': 6,
            'learning_rate': 0.05,
            'early_stopping_rounds': 20,
            'ag_args': {'name_suffix': 'CAT_Custom'}
}]
    },
    verbosity=2
)

Fitting 1 L1 models, fit_strategy="sequential" ...
Fitting model: CatBoostCAT_Custom_BAG_L1 ... Training model for up to 900.00s of the 900.00s of remaining time.
Fitting model: CatBoostCAT_Custom_BAG_L1 ... Training model for up to 900.00s of the 900.00s of remaining time.
	Memory not enough to fit 3 folds in parallel. Will train 1 folds in parallel instead (Estimated 66.17% memory usage per fold, 66.17%/80.00% total).
	Fitting 3 child models (S1F1 - S1F3) | Fitting with ParallelLocalFoldFittingStrategy (1 workers, per: cpus=20, gpus=0, memory=66.17%)
	Memory not enough to fit 3 folds in parallel. Will train 1 folds in parallel instead (Estimated 66.17% memory usage per fold, 66.17%/80.00% total).
	Fitting 3 child models (S1F1 - S1F3) | Fitting with ParallelLocalFoldFittingStrategy (1 workers, per: cpus=20, gpus=0, memory=66.17%)
2025-06-29 23:36:09,551	ERROR services.py:1350 -- Failed to start the dashboard , return code 0
2025-06-29 23:36:09,552	ERROR services.py:1375 -- Error shoul

In [ ]:
predictor = TabularPredictor.load("AutogluonModels/model_fast_20250629_2217")

In [12]:
from autogluon.common import space as ag

predictor.fit_extra(
    time_limit=14400,
    ag_args_ensemble={
        'fold_fitting_strategy': 'sequential_local',
        'num_bag_folds': 3
    },
    fit_weighted_ensemble=True,
    hyperparameters={
        'NN_TORCH': {
            'num_epochs': 30,  # Máximo de epochs (early stopping puede cortar antes)
            'learning_rate': ag.Real(1e-4, 5e-3, log=True),
            'dropout_prob': ag.Real(0.1, 0.5),
            'weight_decay': ag.Real(1e-6, 1e-3, log=True),
            'batch_size': ag.Categorical(256, 512, 1024),
            'hidden_size': ag.Categorical(128, 256, 512),
            'ag_args': {'name_suffix': 'NN_HPO'},
            'ag_args_fit': {
                'num_gpus': 1,
                'max_memory_usage_ratio': 0.4,  # Máximo 40% por trial
            },
            'early_stopping_patience': 5,
        }
    },
    hyperparameter_tune_kwargs={
        'num_trials': 15,
        'scheduler': 'FIFO',
        'searcher': 'random',
    },
    verbosity=2
)

No model was trained during hyperparameter tuning NeuralNetTorchNN_HPO_BAG_L1... Skipping this model.
No base models to train on, skipping auxiliary stack level 2...
TabularPredictor saved. To load, use: predictor = TabularPredictor.load("/home/pablo/Documentos/labo3-2025v/AutogluonModels/model_fast_20250629_2217")


In [7]:
predictor.fit_extra(
    time_limit=600,  
    ag_args_ensemble={'fold_fitting_strategy': 'parallel_local'},
    fit_weighted_ensemble=True,
    hyperparameters={
        'XGB': [{
            'n_estimators': 150,
            'early_stopping_rounds': 10,
            'learning_rate': 0.05,
            'booster': 'gbtree',
            'ag_args': {'name_suffix': 'XGB_Quick'}
        }]
    },
    verbosity=2
)

Fitting 1 L1 models, fit_strategy="sequential" ...
Fitting model: XGBoostXGB_Quick_BAG_L1 ... Training model for up to 600.00s of the 600.00s of remaining time.
Fitting model: XGBoostXGB_Quick_BAG_L1 ... Training model for up to 600.00s of the 600.00s of remaining time.


KeyboardInterrupt: 

In [ ]:
""" import datetime
gc.collect()
# === Configuración general ===
label_column = 'CLASE_DELTA_ZSCORE'
output_path = f"AutogluonModels/model_gbm_{datetime.datetime.now().strftime('%Y%m%d_%H%M')}"

# === Entrenamiento ===
predictor = TabularPredictor(
    label=label_column,
    problem_type='regression',
    eval_metric='mean_absolute_error',
    path=output_path
).fit(
    train_data=df_train,
    tuning_data=df_val,
    use_bag_holdout=True,
    presets='medium_quality',
    num_bag_folds=8,         # Mayor robustez
    num_bag_sets=2,          # Mejor estimación out-of-fold
    auto_stack=True,         # Activar stacking desde el inicio
    time_limit=10800,
    ag_args_ensemble={'fold_fitting_strategy': 'parallel_local'},
    hyperparameters={
        'GBM': [
            {
                'ag_args': {'name_suffix': 'XT'},
                'extra_trees': True,
                'time_limit': 600,
                'early_stopping_rounds': 20,
            },
            {
                'ag_args': {'name_suffix': 'LinearTree'},
                'linear_tree': True,
                'time_limit': 600,
                'early_stopping_rounds': 20,
            },
            {
                'ag_args': {'name_suffix': 'Default'},
                'num_leaves': 31,
                'feature_fraction': 0.8,
                'bagging_fraction': 0.8,
                'time_limit': 600,
                'early_stopping_rounds': 20,
            }
        ]
    }
)
 """

In [ ]:
# # === Cargar modelo ya entrenado ===
# #predictor = TabularPredictor.load("AutogluonModels/model_gbm")

# predictor.fit_extra(
#     time_limit=10800,  # Tiempo total disponible para todos los modelos nuevos
#     ag_args_ensemble={'fold_fitting_strategy': 'parallel_local'},
#     fit_weighted_ensemble=True,  # Activar Weighted Ensemble
#     hyperparameters={
#         'XGB': [{
#             'n_estimators': 1000,
#             'early_stopping_rounds': 25,
#             'learning_rate': 0.03,
#             'booster': 'gbtree',
#             'ag_args': {'name_suffix': 'XGB_Custom'}
#         }],
#         'CAT': [{
#             'ag_args_fit': {'time_limit': 1200},
#             'ag_args': {'name_suffix': 'CAT_Custom'}
#         }],
#         'NN_TORCH': [{
#             'num_epochs': 50,
#             'learning_rate': 0.003,
#             'layers': [1024, 512, 256],
#             'dropout_prob': 0.3,
#             'weight_decay': 1e-5,
#             'batch_size': 2048,
#             'ag_args': {'name_suffix': 'DeepCustom'},
#             'ag_args_fit': {'num_gpus': 1}
#         }]
#     },
#     verbosity=2
# )


In [ ]:
# Comparar MAE y MedAE en el leaderboard
lb = predictor.leaderboard(data = df_val, extra_metrics=['mean_absolute_error', 'median_absolute_error', 'r2'], silent=False)

In [ ]:
ensemble_model = predictor._trainer.load_model(best_model)
weights = getattr(ensemble_model, 'weights', None)
print(f"Best model: {best_model}")
print(weights)


In [ ]:
# Evaluar feature importance sobre el conjunto de validación
importancia = predictor.feature_importance(df_val)

In [ ]:
importancia

✅ Paso 1: Calcular errores de predicción

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Predicciones y error absoluto
df_val['y_true'] = df_val['CLASE_DELTA_ZSCORE']
df_val['y_pred'] = predictor.predict(df_val)
df_val['error_abs'] = abs(df_val['y_true'] - df_val['y_pred'])
df_val['error_signed'] = df_val['y_pred'] - df_val['y_true']


📊 Paso 2: Histogramas de error absoluto y error signado

In [ ]:
plt.figure(figsize=(10, 4))
sns.histplot(df_val['error_abs'], bins=50, kde=True)
plt.title('Distribución del Error Absoluto')
plt.xlabel('|y_pred - y_true|')
plt.ylabel('Frecuencia')
plt.show()

plt.figure(figsize=(10, 4))
sns.histplot(df_val['error_signed'], bins=50, kde=True)
plt.title('Distribución del Error Signado')
plt.xlabel('y_pred - y_true')
plt.ylabel('Frecuencia')
plt.axvline(0, color='red', linestyle='--')
plt.show()


📈 Paso 3: Error vs. Valor real (dispersión)


In [ ]:
plt.figure(figsize=(8, 6))
sns.scatterplot(x=df_val['y_true'], y=df_val['error_signed'], alpha=0.3)
plt.axhline(0, color='red', linestyle='--')
plt.title('Error Signado vs. Valor Real')
plt.xlabel('Valor real')
plt.ylabel('Error (pred - real)')
plt.show()


🧩 Paso 4: Promedio de error por grupo 

In [ ]:
top_errores_producto = df_val.groupby('PRODUCT_ID')['error_abs'].mean().sort_values(ascending=False).head(20)
top_errores_producto.plot(kind='bar', figsize=(10,4), title='Top 20 PRODUCT_ID con mayor error promedio')
plt.ylabel('Error Absoluto Medio')
plt.show()


In [ ]:
top_errores_producto = df_val.groupby('CUSTOMER_ID')['error_abs'].mean().sort_values(ascending=False).head(20)
top_errores_producto.plot(kind='bar', figsize=(10,4), title='Top 20 CUSTOMER_ID con mayor error promedio')
plt.ylabel('Error Absoluto Medio')
plt.show()

In [ ]:
# Combinar entrenamiento + validación
df_full = pd.concat([df_train, df_val], axis=0)
del df_train, df_val
gc.collect()

In [ ]:
# Reentrenar el mejor modelo con TODOS los datos disponibles
predictor_full = predictor.refit_full(train_data=df_full)

In [ ]:
# Verificar los modelos disponibles (el mejor ahora tiene el sufijo '_FULL')
print("Modelos disponibles luego del refit completo:")
print(predictor.leaderboard(silent=True)['model'].tolist())
# Eliminar modelos intermedios para liberar espacio
predictor.delete_models(models_to_keep='best', dry_run=False)

# Confirmar que solo queda el modelo reentrenado
print("\nModelos restantes después de eliminar los intermedios:")
print(predictor.leaderboard(silent=True)['model'].tolist())

# (Opcional) Guardar el predictor final si querés usarlo luego sin volver a cargar todo
predictor.save('./data/modelo_final_autogluon')

# ---  Liberar memoria ---
del df_full
gc.collect()


In [ ]:
# Cargo los datos sobre los que quiero hacer predicciones
df_pred_full = pd.read_parquet('./data/l_vm_completa_normalizada_fe.parquet', engine='fastparquet')
# Dejo solo los datos del periodo 201910 y que A_PREDECIR sea True
# Filtrar solo los datos del periodo 201910 y donde A_PREDECIR sea True
df_pred_full = df_pred_full[
    (df_pred_full['PERIODO'] == 201910) & (df_pred_full['A_PREDECIR'] == True)
].drop(columns=['CLASE_ZSCORE', 'CLASE_DELTA_ZSCORE'])

In [ ]:
# Realizar las predicciones usando el predictor original
predictions = predictor.predict(df_pred_full)
# Agregar las predicciones al DataFrame original
df_pred_full['CLASE_DELTA_ZSCORE'] = predictions

In [ ]:
# Imprimir la lista de columas del DataFrame con las predicciones
print("Columnas del DataFrame con las predicciones:")
print(df_pred_full.columns.tolist())

In [ ]:
# Dernormalizar la columna CLASE_DELTA_ZSCORE
df_pred_full['CLASE_DELTA'] = df_pred_full['CLASE_DELTA_ZSCORE'] * df_pred_full['CLASE_DELTA_STD'] + df_pred_full['CLASE_DELTA_MEAN']
df_pred_full['TN'] = df_pred_full['TN_ZSCORE'] * df_pred_full['TN_STD'] + df_pred_full['TN_MEAN']
# Agregar la columna TN_PREDICT que sea la suma de TN y CLASE_DELTA y si es menor que cero, poner cero
df_pred_full['TN_PREDICT'] = df_pred_full['TN'] + df_pred_full['CLASE_DELTA']
df_pred_full['TN_PREDICT'] = df_pred_full['TN_PREDICT'].clip(lower=0)

In [ ]:
# Generar Dataframe que contenga por cada PRODUCT_ID la suma de TN_PREDICT
df_final = df_pred_full.groupby('PRODUCT_ID').agg({'TN_PREDICT': 'sum'}).reset_index()
df_final = df_final.rename(columns={'PRODUCT_ID': 'product_id', 'TN_PREDICT': 'tn'})
# Guardar el DataFrame df_final en un archivo CSV
df_final.to_csv('./modelos/autoglun_normalizando_clase_delta.csv', index=False)
df_final.shape

# Para instalar AutoGluon con soporte para `autogluon.core.space` en conda, ejecuta:
# 
# conda install -c conda-forge autogluon
# 
# O si prefieres usar pip dentro de tu entorno conda:
# 
# pip install autogluon
# 
# Luego podrás usar:
# from autogluon.core import space as ag